# 1. Import Packages


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import os
import pathlib
from pprint import pprint

#img
import cv2

#pytorch
import torch

from torch import nn
from torch import functional as F
from torch import optim
from torch.utils.data import Dataset,DataLoader,random_split
from torchvision import transforms
from torch.nn import Module
from torchvision import models
from PIL import Image
#dicom
import pydicom

from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.rpn import AnchorGenerator
from torch.utils.data import DataLoader
from albumentations.pytorch.transforms import ToTensorV2
from tqdm.notebook import tqdm
import albumentations as A

#set Device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

# 2.Path

In [ ]:
#-----path-----
#train csv
train_csv_path=pathlib.\
        Path("../input/vinbigdata-chest-xray-abnormalities-detection/train.csv")
sample_sub_path=pathlib.\
        Path("../input/vinbigdata-chest-xray-abnormalities-detection/sample_submission.csv")
#dicom data
train_data_path=pathlib.\
        Path("../input/vinbigdata-chest-xray-abnormalities-detection/train")
test_data_path=pathlib.\
        Path("../input/vinbigdata-chest-xray-abnormalities-detection/test")

#pathの確認
print(pathlib.Path.exists(train_csv_path),
      pathlib.Path.exists(train_data_path),
      pathlib.Path.exists(test_data_path)
     )

In [ ]:
#trainデータのindexリストを取得する。
image_ids=[x for x in train_data_path.iterdir() if x.is_file()]
len(image_ids)


# 3. Read DataFrame : Train Data

In [ ]:
#train csv
df=pd.read_csv(train_csv_path)
df.head()

In [ ]:
#image id部分のみを分割
sample_=pathlib.\
    Path('../input/vinbigdata-chest-xray-abnormalities-detection/train/000434271f63a053c4128a0ba6352c7f.dicom')
#拡張子のみを取得する
print("拡張子；",image_ids[0].suffix)
#拡張子なしのidのみを取得
print("拡張子なしのファイル名取得:",image_ids[0].stem)

In [ ]:
#bounding boxを取得することは可能。
df[df["image_id"]==sample_.stem]

# 4. dataframeの欠損値を修正


### class Name: No finding はデータなしに該当

>0 - Aortic enlargement
1 - Atelectasis
2 - Calcification
3 - Cardiomegaly
4 - Consolidation
5 - ILD
6 - Infiltration
7 - Lung Opacity
8 - Nodule/Mass
9 - Other lesion
10 - Pleural effusion
11 - Pleural thickening
12 - Pneumothorax
13 - Pulmonary fibrosis

### No_findingに該当するには15に指定するか


#### columns;x_min	y_min	x_max	y_maxはNaNになっているため欠損値補完が必要





In [ ]:
print(df.isnull().sum())
#上記意外に欠損値はなさそう。
#bounding boxがない場合Nanとなっているため、穴埋め 
df.fillna(0,inplace=True)

In [ ]:
#classid
"""
>0 - Aortic enlargement
1 - Atelectasis
2 - Calcification
3 - Cardiomegaly
4 - Consolidation
5 - ILD
6 - Infiltration
7 - Lung Opacity
8 - Nodule/Mass
9 - Other lesion
10 - Pleural effusion
11 - Pleural thickening
12 - Pneumothorax
13 - Pulmonary fibrosis
"""
print(df["class_id"].unique())
print(len(df["class_id"].unique()))

In [ ]:
# dicomデータの画像表示例
sample_ids=image_ids[10]
print(sample_ids.stem)
print(sample_ids.suffix)

# 5.1 Dicom Example

In [ ]:
dicom=pydicom.dcmread(sample_ids)
dicom

# 5.2 Normalize Dicom Image

[reference]
https://www.kaggle.com/raddar/popular-x-ray-image-normalization-techniques

In [ ]:
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
from skimage import exposure


#[reference]\
#https://www.kaggle.com/raddar/popular-x-ray-image-normalization-techniques

def read_xray(path, voi_lut = True, fix_monochrome = True):
    dicom = pydicom.read_file(path)
    
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
    
    data = data - np.min(data)
        
    return data

### [1] No-Normalization

In [ ]:
# 1. Non normalization

img = read_xray(str(sample_ids))
plt.figure(figsize=(7,7))
plt.imshow(img, 'gray')
plt.show()

### [2] Histogram normalization
The general idea is to make pixel distribution uniform. This makes X-rays appear a little darker. This generates view, which radiologist would not see in his standard workplace.\
Such normalization is used in popular open-source X-ray datasets, such as CheXpert.

In [ ]:
img = read_xray(str(sample_ids))
img = exposure.equalize_hist(img)
plt.figure(figsize = (7,7))
plt.imshow(img, 'gray')
plt.show()

### [3] CLAHE normalization
This method produces sharper images and is quite often used in chest X-ray research. This generates view, which radiologist would not see in his standard workplace. However, it closely resembles the "bone-enhanced" view in some X-rays done (usually due to broken ribs).



In [ ]:
img = read_xray(str(sample_ids))
img = exposure.equalize_adapthist(img/np.max(img))
plt.figure(figsize = (7,7))
plt.imshow(img, 'gray')
plt.show()

## 6. Data Augmentation and DataSet

>1. multi boudingboxの出力に対応するために train_dataにあるimage_idをまず取得して、そのIDに該当するdataFrameデータを取得する。

>2.targetがマルチ出力になるため、dict形式で出力する

reference
https://www.kaggle.com/pestipeti/vinbigdata-fasterrcnn-pytorch-train

In [ ]:
#[reference]
#https://www.kaggle.com/raddar/popular-x-ray-image-normalization-techniques

def read_xray(path, voi_lut = True, fix_monochrome = True):
    dicom = pydicom.read_file(path)
    
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
    
    data = data - np.min(data)
        
    return data

### 6.1 Data Augmentation

In [ ]:
#Set Image Augumentation
from torchvision import transforms
import albumentations

#transforms.Grayscale(3)

transform=transforms.Compose([
        transforms.ToPILImage(),
        transforms.Grayscale(3),
        transforms.ToTensor(),
        ]) 

###  6.2 Resize Image and BoundingBox

In [ ]:
# Resize Bounding Box

def resize(image, boxes, width, height):
    # 現在の高さと幅を取得しておく
    c_height, c_width = image.shape[:2]
    img = cv2.resize(image, (width, height))
    
    # 圧縮する比率(rate)を計算
    r_width = width / c_width
    r_height = width / c_height
    
    # 比率を使ってBoundingBoxの座標を修正
    new_boxes = []
    for box in boxes:
        x,y,w,h=box
        x = int(x * r_width)
        y = int(y * r_height)
        w = int(w * r_width)
        h = int(h * r_height)
        new_box =[x, y, w, h]
        new_boxes.append(new_box)
    return img, new_boxes

In [ ]:
class My_Dataset(Dataset):
    def __init__(self,df,):
        
        #dataframeを格納する
        self.df = df
        self.image_ids=df["image_id"].unique()
        self.image_dir=pathlib.\
                    Path("../input/vinbigdata-chest-xray-abnormalities-detection/train")
        #columnsを設定する
        self.box_col=["y_min","y_min","x_max","y_max"]
        #transform
        self.transform=transforms.Compose(
            [
            transforms.ToPILImage(),
            transforms.Grayscale(3),
            transforms.ToTensor(),
            ]) 

    def __len__(self):
        return len(self.image_ids)
    
    def __getitem__(self,index,transform=False):
        
        #train_data(dicom)よりrandomでdicomデータを取得
        image_id=self.image_ids[index]
        #print(image_id)
        
        #[dicom_data] #arrayに変換されて出力
        image=read_xray(str(self.image_dir/image_id)+".dicom")

        #Histogram normalization(type:ndarray)
        image = exposure.equalize_hist(image)
        
        
        #-----bboxが複数の可能性あり、複数のデータを取得する必要あり。-----
        records = self.df[(self.df['image_id'] == image_id)]
        records = records.reset_index(drop=True)
        
        if records.loc[0, "class_id"] == 0:
            records = records.loc[[0], :]
        #records = self.df.loc[self.df.image_id == img_path.split('.')[0],:].reset_index(drop = True)
        
        #-----bounding box-----
        boxes = records[self.box_col].values.astype(np.float32)
        #----area-----
        #bbox:[x,y,w,h]とすると、(w-x)*(h-y)で出力される。
        area = (boxes[:,2] - boxes[:,0]) * (boxes[:,3] - boxes[:,1])
        area = area.astype(np.float32)
        
        #----labels-----
        """
        0 - Aortic enlargement,1 - Atelectasis,2 - Calcification,3 - Cardiomegaly,4 - Consolidation,
        5 - ILD,6 - Infiltration,7 - Lung Opacity,8 - Nodule/Mass,9 - Other lesion,10 - Pleural effusion,
        11 - Pleural thickening,12 - Pneumothorax,13 - Pulmonary fibrosis
        15に該当するのはNoneっぽい
        """
        
        labels = torch.tensor(records["class_id"].values, dtype=torch.int64)
        
        # suppose all instances are not crowd
        #iscrowd = torch.zeros((records.shape[0],), dtype=torch.int64)
        
        #元の画像データの画像サイズを取得する
        
        #-----[target]:dict-----
        target = {}
        target['boxes'] = torch.tensor(boxes)
        target['labels'] = labels
        target['image_id'] = torch.tensor([index])
        #target['area'] = torch.tensor(area)
        #target['iscrowd'] = iscrowd
        target["image_row_shape"]=torch.tensor(image.shape)
        target["dicom_id"]=image_id
        
        #Transoformed Image
        #transform
        #image_transformed=self.transform(image.astype(np.float32))
        
        #width,height=[512,512]でresizeする
        width=512
        height=512
        image_resized,boxes_resized=resize(image,boxes,width, height)
        #print("boxes_resized:",boxes_resized)
        target["boxes_resized"]=torch.tensor(boxes_resized)
        
        #transform
        image_transformed=self.transform(image_resized.astype(np.float32))
        
        return image_transformed, target

### Dataloaderで出力する場合

各画像ごとにbounding_boxの数が異なるため、collate_fnを変更する必要あり。

In [ ]:
def collate_fn(batch):
    imgs, targets= list(zip(*batch))
    imgs = torch.stack(imgs)
    #torch.stackをかけると出力ごとに異なるため、torch.stackできない
    #list or　tuppleで返せばうまくいく。
    targets = list(targets)
    #bc = torch.stack(bc)
    return imgs,targets

In [ ]:
train_dataset=My_Dataset(df=df)
train_dataloader=DataLoader(train_dataset,
                            batch_size=2,shuffle=True, 
                            collate_fn= collate_fn)

# Output Sample
image,target =next(iter(train_dataloader))
print("------image-----")
print("image_tensor:",image.shape)
print("-----target-----")
print(target[0])
print(target[1])

## 6.3 Sample Image with Bounding Box

In [ ]:
def draw_boundingbox(target):
    
    #とりあえず上記まで取得しておけば良いか。
    
    #-----image-----
    image_dir=pathlib.\
        Path("../input/vinbigdata-chest-xray-abnormalities-detection/train")
    image_id=target["dicom_id"]
    img=read_xray(str(image_dir/image_id)+".dicom")
    
    #-----bounding box-----
    bboxes=target["boxes"].detach().numpy().astype(int)
    #print("bounding_box:\n",bboxes)
    print("bounding box:",bboxes)
        
    #-----label name-----
    labels=target["labels"].detach().numpy()
    print("label:",labels)
        
    #Plot Image with Bounding Box
    for bbox,label in zip(bboxes,labels):

        x = int(bbox[0])
        y = int(bbox[1])
        w = int(bbox[2])
        h = int(bbox[3])
        color = (0,0,255)
        
        #labelを付与(stringに変換する必要あり)
        cv2.putText(img,str(label), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1.5, color, 3)
        #cv2.rectangleのtuppleにはintを入力する(floatは不可)
        cv2.rectangle(img, (x, y), (w, h), (255,0,0), 2)
    
    plt.figure(num=None, figsize=(5,5), dpi=80, facecolor='w', edgecolor='k')
    plt.imshow(img,cmap="bone")
    plt.show()

In [ ]:
for i in range(len(target)):
    data=target[i]
    draw_boundingbox(data)    

## 6.4 resized_Image and Bounding Box

In [ ]:
def draw_boundingbox_resized(target):
    
    #とりあえず上記まで取得しておけば良いか。
    
    #-----image-----
    image_dir=pathlib.\
        Path("../input/vinbigdata-chest-xray-abnormalities-detection/train")
    image_id=target["dicom_id"]
    img=read_xray(str(image_dir/image_id)+".dicom")
    width,height=512,512
    img=cv2.resize(img, (width, height))
    
    #-----bounding box-----
    bboxes=target["boxes_resized"].detach().numpy().astype(int)
    #print("bounding_box:\n",bboxes)
    print("bounding box:",bboxes)
        
    #-----label name-----
    labels=target["labels"].detach().numpy()
    print("label:",labels)
    #Plot Image with Bounding Box
    for bbox,label in zip(bboxes,labels):

        x = int(bbox[0])
        y = int(bbox[1])
        w = int(bbox[2])
        h = int(bbox[3])
        color = (0,0,255)
        
        #cv2.rectangleのtuppleにはintを入力する(floatは不可)
        cv2.rectangle(img, (x, y), (w, h), (255,0,0), 1)
        
        #labelを付与(stringに変換する必要あり)
        cv2.putText(img,str(label), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1.5, color, 1)
    
    plt.figure(num=None, figsize=(5,5), dpi=80, facecolor='w', edgecolor='k')
    plt.imshow(img,cmap="bone")
    plt.show()

In [ ]:
for i in range(len(target)):
    draw_boundingbox_resized(target[i])

# next: How to Use EfficientDet-pytorch...